In [1]:
import os

In [2]:
param_dir = 'data_20240207'

list_dir = list(os.walk('./' + param_dir))[0][1]
if '.DS_Store' in list_dir:
    list_dir.remove('.DS_Store')
if '0.null.txt' in list_dir:
    list_dir.remove('0.null.txt')
    
list_dir.sort()
    
print(len(list_dir))
list_dir

544


['3D',
 '3M',
 'AAE',
 'ACC',
 'ACCEL',
 'ACDELCO',
 'ACI',
 'ACME AUTO',
 'ADVAN-TECH',
 'ADVICS',
 'AEM INDUCTION',
 'AGILITY',
 'AGS',
 'AIMCO',
 'AIR PRO',
 'AIRAID',
 'AIRQUALITEE',
 'AIRTEX',
 'AISIN',
 'AJUSA',
 'AKEBONO',
 "AL'S LINER",
 'AL-KO',
 'ALBANY',
 'ALLPARTS',
 'ALO',
 'ALTROM',
 'AMERICAN COMPONENTS',
 'AMERICAN REMANUFACTURERS INC.',
 'AMS AUTOMOTIVE',
 'ANCHOR',
 'ANCO',
 'ANSA',
 'AP ~ EASTERN',
 'APEX',
 'API',
 'APWI',
 'ARIES',
 'ARMORMARK',
 'ARNOTT',
 'ARP',
 'ARROW AUTOMOTIVE',
 'ATE',
 'ATP',
 'ATSCO',
 'AUS INJECTION INC.',
 'AUTO 7',
 'AUTO EXTRA',
 'AUTO EXTRA~CHASSIS RITE',
 'AUTO PRIDE',
 'AUTO-TUNE',
 'AUTOBEST',
 'AUTODRIVE',
 'AUTOLINE',
 'AUTOLITE',
 'AUTOMETER',
 'AUTOMOTIVE UNDERCAR PRODUCTS',
 'AUTOSPECIALTY',
 'AUTOSPECIALTY~KELSEY-HAYES',
 'AUTOTECH ENGINEERING',
 'AUTOTECNICA',
 'AUTOTEMP',
 'AUVECO',
 'AVS',
 'B&B',
 "B'LASTER",
 'BALKAMP',
 'BANDO',
 'BB BRAKES',
 'BBB INDUSTRIES',
 'BCA',
 'BECK~ARNLEY',
 'BELDEN',
 'BENDIX',
 'BENTLEY PUB

In [4]:
for param_manufacturer in []:

    from gevent import monkey
    monkey.patch_all(thread=False)
    from gevent.queue import Queue
    import gevent

    import sys
    sys.path.append('../../../00.module')
    import UA
    import Proxy

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_error = pd.DataFrame()

    input_ = pd.read_excel('./' + param_dir + '/' + param_manufacturer + '/3.type.xlsx',
                           header=0,
                           dtype=str).fillna('')

    length = len(input_)

    print('总数量：' + str(length))
    print()

    # = = = = = = = = = = = = = = =

    import requests
    import json

    from bs4 import BeautifulSoup

    from lxml import etree

    # = = = = = = = = = = = = = = =

    import time
    from datetime import datetime

    # = = = = = = = = = = = = = = =

    work = Queue()
    for i in range(length):
        work.put_nowait(i)

    def crawler():
        global output_error

        while not work.empty():
            i = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'
            for _ in range(101):
                try:
                    resp = requests.get(input_.loc[i, 'Url'],
                                        headers=UA.get_User_Agent_Requests(),
                                        proxies=Proxy.get_Proxy_Requests()).text

                    # = = = = = = = = = = = = = = =

                    soup = BeautifulSoup(resp, 'lxml')

                    # = = = = = = = = = = = = = = =

                    html = etree.HTML(str(soup))
                    
                    title = html.xpath('//div[contains(@style, "display: inline-block; padding: 0em 5em 0em 5em;")]/text()')[0]
                    
                    # = = = = = = = = = = = = = = =
                    
                    if input_.loc[i, 'Manufacturer'] not in title or input_.loc[i, 'Type'] not in title or 'Part Numbers' not in title:
                        continue
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_part = [json.loads(part) for part in html.xpath('//div[@class="nchildren"]/div[@class="ranavnode"]/input[contains(@id, "jsn")]/@value')]
                    
                    # = = = = = = = = = = = = = = =

                    list_part_number = [part['partnumber'].strip() for part in list_part]
                    
                    # = = = = = = = = = = = = = = =

                    crawler_status = 'ok'

                    break

                except:
                    time.sleep(0.3)
                    continue

            if crawler_status == 'ok':
                df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                        'No.': int(input_.loc[i, 'No.']),
                                        'Make': '',
                                        'Year': '',
                                        'Model': '',
                                        'Engine': '',
                                        'Car_Code': '',
                                        'Category': input_.loc[i, 'Category'],
                                        'Subcategory ': '',
                                        'Type': input_.loc[i, 'Type'],
                                        'Type_Code': input_.loc[i, 'Type_Code'],
                                        'Tab_Url': input_.loc[i, 'Url'],
                                        'Part_Index': [(index+1) for index in range(len(list_part))],
                                        'Tag': '',
                                        'Manufacturer': input_.loc[i, 'Manufacturer'],
                                        'Part_Number': list_part_number,
                                        'Vehcile': '',
                                        'OE': '',
                                        'Note_1': '',
                                        'Note_2': '',
                                        'Pic': '',
                                        'Url': '',
                                        'Info': '',
                                        'Src': '',
                                        'Part_Type': '',
                                        'Part_Key': '',
                                        'JOIN_MPN': [input_.loc[i, 'Manufacturer']+';'+part_number for part_number in list_part_number]})

                df_temp.to_excel('./' + param_dir + '/' + param_manufacturer + '/4.part_number/' + input_.loc[i, 'Type'].replace('/', '~') + '.xlsx', index=False)
            else:
                df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                         'No.': int(input_.loc[i, 'No.']),
                                         'Manufacturer': input_.loc[i, 'Manufacturer'],
                                         'Category': input_.loc[i, 'Category'],
                                         'Type': input_.loc[i, 'Type'],
                                         'Type_Code': input_.loc[i, 'Type_Code'],
                                         'Url': input_.loc[i, 'Url']}])

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            print(input_.loc[i, 'Manufacturer'] + ' - ' + input_.loc[i, 'Category'] + ' - ' + input_.loc[i, 'Type'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(20):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print()
    print('爬虫输出中')
    if len(output_error) != 0:
        output_error = output_error.drop_duplicates(ignore_index=True)
        output_error = output_error.sort_values(by=['No.', 'Category', 'Type'], ascending=[True, True, True])
        output_error.to_excel('./' + param_dir + '/' + param_manufacturer + '/part_number_error.xlsx', index=False)
        print()
        print('爬虫存在error')
    print()
    print('爬虫结束')

总数量：111

ELRING - Drivetrain - Axle Shaft Seal  <->  [ok] - 剩余数量：91 - 当前时间：10:26:01
ELRING - Engine - Auxiliary Shaft Seal  <->  [ok] - 剩余数量：90 - 当前时间：10:26:01
ELRING - Cooling System - Coolant Hose Seal  <->  [ok] - 剩余数量：89 - 当前时间：10:26:01
ELRING - Engine - Camshaft Gasket  <->  [ok] - 剩余数量：88 - 当前时间：10:26:01
ELRING - Cooling System - Coolant Pipe Gasket  <->  [ok] - 剩余数量：87 - 当前时间：10:26:01
ELRING - Engine - Crankcase Cover Gasket Set  <->  [ok] - 剩余数量：86 - 当前时间：10:26:02
ELRING - Engine - Camshaft Seal  <->  [ok] - 剩余数量：85 - 当前时间：10:26:02
ELRING - Engine - Crankshaft Seal  <->  [ok] - 剩余数量：84 - 当前时间：10:26:02
ELRING - Cooling System - Coolant Pipe O-Ring  <->  [ok] - 剩余数量：83 - 当前时间：10:26:02
ELRING - Cooling System - Water Pump Gasket  <->  [ok] - 剩余数量：82 - 当前时间：10:26:02
ELRING - Engine - Camshaft Plug  <->  [ok] - 剩余数量：81 - 当前时间：10:26:02
ELRING - Engine - Circular / Semi-Circular Plug  <->  [ok] - 剩余数量：80 - 当前时间：10:26:02
ELRING - Engine - Conversion / Lower Gasket Set  <->  [ok] - 剩余数量

ELRING - Transmission-Automatic - Fluid Pan Gasket  <->  [ok] - 剩余数量：0 - 当前时间：10:26:17
ELRING - Transmission-Manual - Manual Shaft Seal  <->  [ok] - 剩余数量：0 - 当前时间：10:26:18
ELRING - Transmission-Manual - Main / Output Shaft Seal  <->  [ok] - 剩余数量：0 - 当前时间：10:26:18
ELRING - Transmission-Manual - Manual Transmission Seal  <->  [ok] - 剩余数量：0 - 当前时间：10:26:18
ELRING - Transmission-Automatic - Input Shaft Seal  <->  [ok] - 剩余数量：0 - 当前时间：10:26:21
ELRING - Engine - Cylinder Head Gasket  <->  [ok] - 剩余数量：0 - 当前时间：10:26:22
ELRING - Transmission-Manual - Shift Shaft Seal  <->  [ok] - 剩余数量：0 - 当前时间：10:26:25
ELRING - Engine - Oil Pan Gasket  <->  [ok] - 剩余数量：0 - 当前时间：10:26:41
ELRING - Engine - Oil Strainer Gasket  <->  [ok] - 剩余数量：0 - 当前时间：10:29:23

爬虫输出中

爬虫结束
总数量：3

EMPIRE - Steering - Steering Damper Bellows  <->  [ok] - 剩余数量：0 - 当前时间：10:29:25
EMPIRE - Drivetrain - CV Axle  <->  [ok] - 剩余数量：0 - 当前时间：10:29:25
EMPIRE - Drivetrain - CV Joint Boot  <->  [ok] - 剩余数量：0 - 当前时间：10:29:25

爬虫输出中

爬虫结束
总数量：

ENGINETECH - Engine - Engine Rebuild Kit  <->  [ok] - 剩余数量：24 - 当前时间：10:29:42
ENGINETECH - Engine - Oil Pump Shaft  <->  [ok] - 剩余数量：23 - 当前时间：10:29:42
ENGINETECH - Engine - Exhaust Valve  <->  [ok] - 剩余数量：22 - 当前时间：10:29:43
ENGINETECH - Engine - Oil Pump Pickup Tube / Screen  <->  [ok] - 剩余数量：21 - 当前时间：10:29:43
ENGINETECH - Engine - Intake Manifold Gasket  <->  [ok] - 剩余数量：20 - 当前时间：10:29:43
ENGINETECH - Engine - Intake Plenum Gasket  <->  [ok] - 剩余数量：19 - 当前时间：10:29:43
ENGINETECH - Engine - Piston  <->  [ok] - 剩余数量：18 - 当前时间：10:29:43
ENGINETECH - Engine - Timing Belt & Component Kit  <->  [ok] - 剩余数量：17 - 当前时间：10:29:44
ENGINETECH - Engine - Rocker Arm  <->  [ok] - 剩余数量：16 - 当前时间：10:29:44
ENGINETECH - Engine - Timing Chain Guide / Damper  <->  [ok] - 剩余数量：15 - 当前时间：10:29:44
ENGINETECH - Engine - Piston Ring  <->  [ok] - 剩余数量：14 - 当前时间：10:29:45
ENGINETECH - Engine - Piston Pin Bushing  <->  [ok] - 剩余数量：13 - 当前时间：10:29:45
ENGINETECH - Engine - Push Rod  <->  [ok] - 剩余数量：12 - 当前时间：10:29:

FAB FOURS - Body & Lamp Assembly - Step / Nerf Bar  <->  [ok] - 剩余数量：0 - 当前时间：10:31:14
FAB FOURS - Body & Lamp Assembly - Grille Guard / Bull Bar  <->  [ok] - 剩余数量：0 - 当前时间：10:31:46

爬虫输出中

爬虫结束
总数量：38

FACET - Cooling System - Thermostat  <->  [ok] - 剩余数量：18 - 当前时间：10:31:47
FACET - Electrical-Switch & Relay - Radiator Fan Switch  <->  [ok] - 剩余数量：17 - 当前时间：10:31:47
FACET - Electrical-Switch & Relay - Back Up Lamp Switch  <->  [ok] - 剩余数量：16 - 当前时间：10:31:47
FACET - Cooling System - Thermostat Housing / Water Outlet  <->  [ok] - 剩余数量：15 - 当前时间：10:31:47
FACET - Electrical - Speed Sensor  <->  [ok] - 剩余数量：14 - 当前时间：10:31:47
FACET - Electrical-Switch & Relay - Cruise Control Release Switch  <->  [ok] - 剩余数量：13 - 当前时间：10:31:48
FACET - Electrical-Switch & Relay - Clutch Pedal Position / Starter Safety Switch  <->  [ok] - 剩余数量：12 - 当前时间：10:31:48
FACET - Electrical-Switch & Relay - Neutral Safety Switch / Range Sensor  <->  [ok] - 剩余数量：11 - 当前时间：10:31:48
FACET - Cooling System - Coolant Pipe O

FAIRCHILD - Body & Lamp Assembly - Vent Window Seal / Weatherstrip  <->  [ok] - 剩余数量：31 - 当前时间：10:32:01
FAIRCHILD - Body & Lamp Assembly - Truck Bed Side Seal / Weatherstrip  <->  [ok] - 剩余数量：30 - 当前时间：10:32:01
FAIRCHILD - Body & Lamp Assembly - Weatherstrip Installation Kit  <->  [ok] - 剩余数量：29 - 当前时间：10:32:01
FAIRCHILD - Body & Lamp Assembly - Windshield Glass  <->  [ok] - 剩余数量：28 - 当前时间：10:32:01
FAIRCHILD - Body & Lamp Assembly - Weatherstrip Retainer / Clip  <->  [ok] - 剩余数量：27 - 当前时间：10:32:01
FAIRCHILD - Body & Lamp Assembly - Window Molding  <->  [ok] - 剩余数量：26 - 当前时间：10:32:02
FAIRCHILD - Body & Lamp Assembly - Trunk Bumper  <->  [ok] - 剩余数量：25 - 当前时间：10:32:02
FAIRCHILD - Body & Lamp Assembly - Windlace  <->  [ok] - 剩余数量：24 - 当前时间：10:32:02
FAIRCHILD - Body & Lamp Assembly - Window Retainer / Clip  <->  [ok] - 剩余数量：23 - 当前时间：10:32:02
FAIRCHILD - Body & Lamp Assembly - Weatherstrip Kit  <->  [ok] - 剩余数量：22 - 当前时间：10:32:02
FAIRCHILD - Exhaust & Emission - Positive Crankcase Ventilat

...

ÜRO PARTS - Engine - Cylinder Head Nut  <->  [ok] - 剩余数量：415 - 当前时间：14:25:00
ÜRO PARTS - Electrical-Switch & Relay - Power Window Switch  <->  [ok] - 剩余数量：414 - 当前时间：14:25:00
ÜRO PARTS - Engine - Expansion Plug  <->  [ok] - 剩余数量：413 - 当前时间：14:25:00
ÜRO PARTS - Engine - Engine Compartment Seal  <->  [ok] - 剩余数量：412 - 当前时间：14:25:00
ÜRO PARTS - Engine - Camshaft Oiler Kit  <->  [ok] - 剩余数量：411 - 当前时间：14:25:01
ÜRO PARTS - Engine - Intake Manifold Actuator  <->  [ok] - 剩余数量：410 - 当前时间：14:25:01
ÜRO PARTS - Engine - Intake Boot  <->  [ok] - 剩余数量：409 - 当前时间：14:25:01
ÜRO PARTS - Engine - Cylinder Head Bolt Washer  <->  [ok] - 剩余数量：408 - 当前时间：14:25:01
ÜRO PARTS - Engine - Intake Manifold Cover  <->  [ok] - 剩余数量：407 - 当前时间：14:25:01
ÜRO PARTS - Engine - Motor Mount  <->  [ok] - 剩余数量：406 - 当前时间：14:25:02
ÜRO PARTS - Engine - Motor Mount Bushing  <->  [ok] - 剩余数量：405 - 当前时间：14:25:02
ÜRO PARTS - Engine - Oil Cooler  <->  [ok] - 剩余数量：404 - 当前时间：14:25:03
ÜRO PARTS - Engine - Oil Cooler Seal  <->  [ok] -

ÜRO PARTS - Exhaust & Emission - Manifold Pressure (MAP) Sensor Vacuum Hose  <->  [ok] - 剩余数量：315 - 当前时间：14:25:19
ÜRO PARTS - Exhaust & Emission - Manifold Pressure (MAP) Sensor  <->  [ok] - 剩余数量：314 - 当前时间：14:25:19
ÜRO PARTS - Exhaust & Emission - Positive Crankcase Ventilation (PCV) Grommet  <->  [ok] - 剩余数量：313 - 当前时间：14:25:20
ÜRO PARTS - Exhaust & Emission - Oil Separator Hose  <->  [ok] - 剩余数量：312 - 当前时间：14:25:20
ÜRO PARTS - Exhaust & Emission - Oil Separator  <->  [ok] - 剩余数量：311 - 当前时间：14:25:20
ÜRO PARTS - Exhaust & Emission - Positive Crankcase Ventilation (PCV) Hose Nipple  <->  [ok] - 剩余数量：310 - 当前时间：14:25:20
ÜRO PARTS - Exhaust & Emission - Exhaust Mount  <->  [ok] - 剩余数量：309 - 当前时间：14:25:21
ÜRO PARTS - Exhaust & Emission - Knock / Detonation Sensor  <->  [ok] - 剩余数量：308 - 当前时间：14:25:21
ÜRO PARTS - Exhaust & Emission - Positive Crankcase Ventilation (PCV) Hose  <->  [ok] - 剩余数量：307 - 当前时间：14:25:21
ÜRO PARTS - Exhaust & Emission - Vacuum Hose  <->  [ok] - 剩余数量：306 - 当前时间：14:2

ÜRO PARTS - Heat & Air Conditioning - Heater Core  <->  [ok] - 剩余数量：227 - 当前时间：14:25:37
ÜRO PARTS - Heat & Air Conditioning - Cabin Air Temperature Sensor  <->  [ok] - 剩余数量：226 - 当前时间：14:25:37
ÜRO PARTS - Heat & Air Conditioning - Heater / Evaporator Case Drain Hose  <->  [ok] - 剩余数量：225 - 当前时间：14:25:37
ÜRO PARTS - Heat & Air Conditioning - Heater Hose Coolant Tube  <->  [ok] - 剩余数量：224 - 当前时间：14:25:37
ÜRO PARTS - Heat & Air Conditioning - Heater Hose Retainer  <->  [ok] - 剩余数量：223 - 当前时间：14:25:38
ÜRO PARTS - Heat & Air Conditioning - Heater Valve O-Ring  <->  [ok] - 剩余数量：222 - 当前时间：14:25:38
ÜRO PARTS - Heat & Air Conditioning - Heater Hose Connector  <->  [ok] - 剩余数量：221 - 当前时间：14:25:38
ÜRO PARTS - Ignition - Distributor Primary Lead Wire  <->  [ok] - 剩余数量：220 - 当前时间：14:25:38
ÜRO PARTS - Fuel & Air - Throttle Body Gasket  <->  [ok] - 剩余数量：219 - 当前时间：14:25:38
ÜRO PARTS - Hoses/Lines & Clamps - Heater Hose O-Ring  <->  [ok] - 剩余数量：218 - 当前时间：14:25:38
ÜRO PARTS - Heat & Air Conditioning 

ÜRO PARTS - Interior - Window Regulator  <->  [ok] - 剩余数量：129 - 当前时间：14:25:55
ÜRO PARTS - Interior - Transmission Shift Handle  <->  [ok] - 剩余数量：128 - 当前时间：14:25:56
ÜRO PARTS - Interior - Window Guide  <->  [ok] - 剩余数量：127 - 当前时间：14:25:56
ÜRO PARTS - Interior - Window Regulator Rivet  <->  [ok] - 剩余数量：126 - 当前时间：14:25:56
ÜRO PARTS - Interior - Window Crank Handle  <->  [ok] - 剩余数量：125 - 当前时间：14:25:56
ÜRO PARTS - Interior - Window Motor  <->  [ok] - 剩余数量：124 - 当前时间：14:25:57
ÜRO PARTS - Interior - Window Regulator & Motor Assembly  <->  [ok] - 剩余数量：123 - 当前时间：14:25:57
ÜRO PARTS - Interior - Window Regulator Guide  <->  [ok] - 剩余数量：122 - 当前时间：14:25:57
ÜRO PARTS - Interior - Trunk or Cargo Area Lamp Assembly  <->  [ok] - 剩余数量：121 - 当前时间：14:25:57
ÜRO PARTS - Interior - Window Regulator Clip  <->  [ok] - 剩余数量：120 - 当前时间：14:25:57
ÜRO PARTS - Exhaust & Emission - Positive Crankcase Ventilation (PCV) Valve  <->  [ok] - 剩余数量：119 - 当前时间：14:25:57
ÜRO PARTS - Interior - Windshield Washer Switch Lev

ÜRO PARTS - Transmission-Manual - Clutch Fork  <->  [ok] - 剩余数量：32 - 当前时间：14:26:13
ÜRO PARTS - Transmission-Manual - Clutch Cable Adjuster  <->  [ok] - 剩余数量：31 - 当前时间：14:26:14
ÜRO PARTS - Transmission-Manual - Clutch Cable Sleeve  <->  [ok] - 剩余数量：30 - 当前时间：14:26:14
ÜRO PARTS - Transmission-Manual - Clutch Release Bearing Clip  <->  [ok] - 剩余数量：29 - 当前时间：14:26:14
ÜRO PARTS - Transmission-Manual - Clutch Bushing  <->  [ok] - 剩余数量：28 - 当前时间：14:26:14
ÜRO PARTS - Transmission-Manual - Clutch Push Rod  <->  [ok] - 剩余数量：27 - 当前时间：14:26:14
ÜRO PARTS - Transmission-Manual - Clutch Push Rod Lever  <->  [ok] - 剩余数量：26 - 当前时间：14:26:15
ÜRO PARTS - Transmission-Manual - Clutch Pivot Ball  <->  [ok] - 剩余数量：25 - 当前时间：14:26:15
ÜRO PARTS - Transmission-Manual - Clutch Fork Shaft  <->  [ok] - 剩余数量：24 - 当前时间：14:26:15
ÜRO PARTS - Transmission-Manual - Clutch Master Cylinder  <->  [ok] - 剩余数量：23 - 当前时间：14:26:15
ÜRO PARTS - Transmission-Manual - Main / Output Shaft Seal  <->  [ok] - 剩余数量：22 - 当前时间：14:26:15


In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_error = pd.DataFrame()

input_ = pd.read_excel('./part_number_error.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_error

    while not work.empty():
        i = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(input_.loc[i, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')

                # = = = = = = = = = = = = = = =

                html = etree.HTML(str(soup))

                title = html.xpath('//div[contains(@style, "display: inline-block; padding: 0em 5em 0em 5em;")]/text()')[0]

                # = = = = = = = = = = = = = = =

                if input_.loc[i, 'Manufacturer'] not in title or input_.loc[i, 'Type'] not in title or 'Part Numbers' not in title:
                    continue

                # = = = = = = = = = = = = = = =

                list_part = [json.loads(part) for part in html.xpath('//div[@class="nchildren"]/div[@class="ranavnode"]/input[contains(@id, "jsn")]/@value')]

                # = = = = = = = = = = = = = = =

                list_part_number = [part['partnumber'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                break

            except:
                time.sleep(0.3)
                continue

        if crawler_status == 'ok':
            df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                    'No.': int(input_.loc[i, 'No.']),
                                    'Make': '',
                                    'Year': '',
                                    'Model': '',
                                    'Engine': '',
                                    'Car_Code': '',
                                    'Category': input_.loc[i, 'Category'],
                                    'Subcategory ': '',
                                    'Type': input_.loc[i, 'Type'],
                                    'Type_Code': input_.loc[i, 'Type_Code'],
                                    'Tab_Url': input_.loc[i, 'Url'],
                                    'Part_Index': [(index+1) for index in range(len(list_part))],
                                    'Tag': '',
                                    'Manufacturer': input_.loc[i, 'Manufacturer'],
                                    'Part_Number': list_part_number,
                                    'Vehcile': '',
                                    'OE': '',
                                    'Note_1': '',
                                    'Note_2': '',
                                    'Pic': '',
                                    'Url': '',
                                    'Info': '',
                                    'Src': '',
                                    'Part_Type': '',
                                    'Part_Key': '',
                                    'JOIN_MPN': [input_.loc[i, 'Manufacturer']+';'+part_number for part_number in list_part_number]})

            df_temp.to_excel('./' + param_dir + '/' + input_.loc[i, 'Manufacturer'].replace('/', '~') + '/4.part_number/' + input_.loc[i, 'Type'].replace('/', '~') + '.xlsx', index=False)
        else:
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Manufacturer': input_.loc[i, 'Manufacturer'],
                                     'Category': input_.loc[i, 'Category'],
                                     'Type': input_.loc[i, 'Type'],
                                     'Type_Code': input_.loc[i, 'Type_Code'],
                                     'Url': input_.loc[i, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Manufacturer'] + ' - ' + input_.loc[i, 'Category'] + ' - ' + input_.loc[i, 'Type'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.', 'Category', 'Type'], ascending=[True, True, True])
    output_error.to_excel('./part_number_error_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')